DATAFRAME_STEAM_GAMES

Como primer paso se deben importar las respectivas librerias a usar

In [1]:
# En este caso se van a importan las librerias que se van a utilizar en este proyecto
import gzip
import ast
import pandas as pd
import nltk

Como siguiente paso se define donde esta la ruta del archivo JSON comprimido donde esta alojada la informacion de data_steam_game 

In [2]:
# Aqui con la siguiente linea de codigo se va leer el archivo json
df_steam_games = pd.read_json('../datasets/steam_games.json.gz', compression='gzip', lines=True)
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


Aqui vamos a mostrae las primeras 5 filas del DataFrame steam_games, lo que te dará una idea de cómo se ven los datos y qué tipo de información contienen. 

In [3]:
df_steam_games.head()


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Para el proceso de creación de funciones para los endpoints, es fundamental identificar las columnas que se utilizarán. Una vez identificadas, se inicia el proceso de limpieza de datos. Una de las primeras tareas en este proceso es eliminar las filas que contienen valores nulos, ya que estos pueden afectar negativamente el rendimiento de los modelos o la precisión de los resultados.

Por ende se identifican las columnas y se hace la respectiva eliminación de filas con valores nulos, para asegurar que los datos con los que se trabajará estén completos y sean consistentes. Esto es importante para evitar problemas durante el análisis y para garantizar la calidad de los resultados finales.

En las siguientes lineas del código primero elimina las filas que están completamente compuestas por valores nulos en el DataFrame steam_games, y luego calcula la cantidad de valores nulos que quedan en cada columna después de la eliminación. Esto es útil para verificar cuántos valores nulos quedan en el conjunto de datos después de realizar la limpieza inicial.

In [4]:
df_steam_games.dropna(how="all", inplace=True  )
df_steam_games.isnull().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

Para cambiar el tipo de dato en la columna "id", primero es necesario identificar si esta columna contiene valores nulos. esto nos ayuda a mostrar solo las filas donde el valor en la columna "id" es nulo. Al hacerlo, se identifican las filas específicas que contienen valores nulos en la columna "id". Esta información es crucial antes de intentar cambiar el tipo de dato de la columna, ya que los valores nulos pueden afectar la conversión de tipos de datos. Para hacerlo, se utiliza la siguiente línea de código:

In [5]:
df_steam_games[df_steam_games["id"].isnull()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


Se identificó que la fila 88384 solo contiene la URL, la cual no parece ser relevante para el análisis de datos. Por esta razón, se procede a eliminar esta fila del DataFrame, ya que no aporta información útil para el análisis.

Por otro lado, para la fila 119271, que contiene casi toda la información necesaria, se realizará un filtrado para identificar si existe un duplicado de esta fila en el DataFrame. Dado que esta fila parece ser importante y completa, la presencia de un duplicado podría indicar un error en los datos o una repetición que debe ser investigada. Identificar y tratar los duplicados es importante para mantener la integridad y la precisión de los datos antes de realizar análisis posteriores.

In [6]:
df_steam_games[(df_steam_games['title']=="Batman: Arkham City - Game of the Year Edition") & (df_steam_games['developer']=="Rocksteady Studios,Feral Interactive (Mac)")]


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
89378,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,"Rocksteady Studios,Feral Interactive (Mac)"
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


Durante el análisis de los datos, se descubrió que existe otra fila que contiene la misma información que la fila 119271, pero esta vez asociada a un "id". Dado que la presencia de la columna "id" es importante para identificar de manera única cada entrada en el conjunto de datos, se decidió eliminar la fila que no tiene un valor válido en la columna "id". Esto se hace para evitar redundancias y garantizar la integridad de los datos en el análisis posterior.

In [7]:
df_steam_games.dropna(subset=['id'], inplace=True) #elimina filas de la columna id que son null

Luego se hace la respectiva verificacion de que la columna "id" no tenga duplicados.

In [8]:
df_steam_games[df_steam_games.duplicated(subset=['id'], keep=False)]


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


Dado que la columna "id" no debería tener valores duplicados, se tomó la decisión de eliminar una de las filas que tiene un "id" duplicado. Esto se hace para mantener la integridad de los datos y asegurar que cada "id" sea único en el conjunto de datos, lo que facilita su uso como identificador único para cada entrada.

In [9]:
df_steam_games.drop_duplicates(subset="id", inplace=True, keep="last")

Después de realizar las operaciones de limpieza y eliminación de duplicados, el conjunto de datos está listo para que se pueda cambiar el tipo de dato de la columna "id". Esto significa que se han realizado las preparaciones necesarias para que la conversión de tipo de dato sea segura y precisa, lo que contribuye a la calidad y la coherencia de los datos en el análisis subsiguiente.

In [10]:
df_steam_games["id"] = df_steam_games["id"].astype(int)

Realizamos la respectiva verificacion de nulos en el DataFrame Steam_games, con la siguiente linea devuelve una serie que muestra la cantidad de valores nulos en cada columna del DataFrame steam_games. Esta información es útil para comprender la calidad de los datos y determinar si se necesitan más operaciones de limpieza o imputación de valores nulos antes de realizar análisis o modelado de datos.

In [11]:
df_steam_games.isnull().sum()

publisher       8051
genres          3282
app_name           1
title           2049
url                0
release_date    2066
tags             162
reviews_url        0
specs            669
price           1377
early_access       0
id                 0
developer       3298
dtype: int64

Para verificar los valores nulos en la columna "genres", que será utilizada más adelante en las funciones, se comparó con la columna "tags", que contiene información similar. Esto se hizo para identificar cuántos valores nulos quedarían si se cruzara la información de ambas columnas.

En el siguiente código la función shape devuelve las dimensiones del DataFrame resultante, es decir, el número de filas y columnas que cumplen con la condición especificada. Con ello vamos a proporcionar una forma de determinar cuántas filas tienen valores nulos en las columnas "genres" y "tags" al mismo tiempo, lo que puede ser útil para evaluar la integridad de los datos antes de utilizar estas columnas en análisis posteriores.

In [12]:
df_steam_games[(df_steam_games["genres"].isnull()) & (df_steam_games["tags"].isnull())].shape

(138, 13)

Los valores nulos en la columna "genres" se reemplazaron por los datos de la columna "tags". Esto se hizo porque, en algunas filas, el contenido de ambas columnas es similar o idéntico, con pequeñas diferencias en el orden de los elementos o en la forma en que están presentados. Al hacer este reemplazo, se busca aprovechar la información disponible en la columna "tags" para completar o mejorar los datos faltantes en la columna "genres", lo que puede ser útil para análisis posteriores.

In [13]:
# Aqui se va realizar la creacion de un conjunto de géneros únicos 
genre_dist = set(item for val in df_steam_games['genres'].dropna() for item in val)

# Para luego se pueda realizar el proceso de filtrado de las etiquetas 'tags' 
df_steam_games['tags'] = df_steam_games['tags'].apply(lambda x: [item for item in x if item in genre_dist] if isinstance(x, list) else x) 

# para despues rellenar los valores nulos en la columna 'genres' 
df_steam_games['genres'].fillna(df_steam_games['tags'], inplace=True)

def tags_to_genres(row): 
    ''' Esta función añade valores de 'tags' a 'genres' que no estén ya'''
    genres = row['genres']
    tags = row['tags']
    if isinstance(tags, list) and isinstance(genres, list):
        for tag in tags:
            if tag not in genres:
                genres.append(tag)
    return genres



In [14]:
#Aqui se utiliza la funcion
df_steam_games['genres'] = df_steam_games.apply(lambda row: tags_to_genres(row), axis=1)

In [15]:
# Se elimina la columna 'tags'
df_steam_games.drop(columns=['tags'], inplace=True)

Decidimos eliminar los valores nulos de la columna "genres" ya que no podemos manejarlos adecuadamente para nuestro análisis. Al eliminar estos valores nulos, aseguramos que los datos en la columna "genres" estén completos y listos para su uso en análisis posteriores. Tambien Visualizamos el % de nulos.

In [16]:
(df_steam_games['genres'].isnull().sum() / df_steam_games.shape[0])*100 #Con esto podemos visualizar el % de nulos

0.42947840159342715

En la siguiente línea de código nos proporciona el recuento total de valores nulos en la columna "genres" del DataFrame steam_games, lo que es útil para evaluar la calidad y la integridad de los datos antes de realizar análisis posteriores.

In [17]:
df_steam_games['genres'].isnull().sum()

138

Se elimna los nulos que quedaron en la columna "genres"

In [18]:
df_steam_games_clean = df_steam_games[df_steam_games['genres'].notnull()]

En el siguiente codigo vamos a revisar la columna release_date que es necesaria para hacer la primer función. Esto es útil para entender la distribución de las fechas de lanzamiento y puede ser útil para análisis posteriores, como la creación de una función que requiera esta información.

In [19]:
df_steam_games_clean["release_date"].value_counts() # Obtenemos la cantidad de veces que aparece cada fecha de lanzamiento en la columna "release_date"

release_date
2012-10-16            100
2017-08-31             92
2017-09-26             89
2017-06-21             82
2017-07-25             78
                     ... 
2017 Q4                 1
Play Beta in demo!      1
1970-07-15              1
1975-12-31              1
2018-10-01              1
Name: count, Length: 3579, dtype: int64

Para extraer el año que vamos a utilizar en la función, creamos una nueva columna llamada "year" que contiene únicamente el año de la columna "release_date". Esto nos permitirá trabajar específicamente con los años de lanzamiento de los juegos en nuestro análisis.

In [20]:
# Definición de la función para extraer el año de una fecha
def anio(fecha):
    try:
       # Intenta convertir la fecha a un objeto datetime y extraer el año.
        año = pd.to_datetime(fecha, errors='coerce').year
       # Si se obtiene un año válido, se convierte a entero, de lo contrario,
        return int(año) if not pd.isnull(año) else None
    except:
         # En caso de error, devuelve None
        return None


In [21]:

# Aplicar la función para crear la nueva columna 'year' con los años extraídos
df_steam_games_clean['year'] = df_steam_games_clean['release_date'].apply(anio)

C:\Users\Willc\AppData\Local\Temp\ipykernel_20260\586216625.py:5: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  año = pd.to_datetime(fecha, errors='coerce').year
C:\Users\Willc\AppData\Local\Temp\ipykernel_20260\678893542.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_clean['year'] = df_steam_games_clean['release_date'].apply(anio)


Verificamos la proporción de valores nulos en la columna "year", ya que actualmente no hemos realizado ningún tratamiento para estos valores.

In [22]:
df_steam_games_clean['year'].isnull().sum()

2239

In [23]:
(df_steam_games_clean['year'].isnull().sum() / df_steam_games_clean.shape[0])*100 #porcentaje de nulos

6.998187160092517

Se verifica la fila null del la columna title

In [24]:
df_steam_games_clean[df_steam_games_clean['title'].isnull()]

,publisher,genres,app_name,title,url,release_date,reviews_url,specs,price,early_access,id,developer,year
88314,None,"[Action, Indie, Casual, Sports]",Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570,None,NaN
88321,None,[Casual],Icarus Six Sixty Six,None,http://store.steampowered.com/app/724910/Icaru...,None,http://steamcommunity.com/app/724910/reviews/?...,"[Single-player, HTC Vive, Tracked Motion Contr...",Free,0.0,724910,None,NaN
88329,None,"[Early Access, Indie]",After Life VR,None,http://store.steampowered.com/app/772590/After...,None,http://steamcommunity.com/app/772590/reviews/?...,"[Single-player, HTC Vive, Tracked Motion Contr...",4.99,1.0,772590,None,NaN
88330,None,"[Early Access, Action, Adventure, Indie, Casual]",Kitty Hawk,None,http://store.steampowered.com/app/640250/Kitty...,None,http://steamcommunity.com/app/640250/reviews/?...,"[Single-player, Steam Leaderboards, HTC Vive, ...",2.99,1.0,640250,None,NaN
88332,None,"[Early Access, Strategy, Action, Indie, Casual]",Mortars VR,None,http://store.steampowered.com/app/711440/Morta...,None,http://steamcommunity.com/app/711440/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.99,1.0,711440,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120383,None,"[Action, Massively Multiplayer, Strategy]",Tank of War-VR,None,http://store.steampowered.com/app/745900/Tank_...,None,http://steamcommunity.com/app/745900/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",19.99,0.0,745900,None,NaN
120386,None,"[Casual, Simulation]",Flappy Arms,None,http://store.steampowered.com/app/764110/Flapp...,None,http://steamcommunity.com/app/764110/reviews/?...,"[Single-player, Steam Leaderboards, HTC Vive, ...",1.99,0.0,764110,None,NaN
120387,None,"[Early Access, Casual]",SpaceWalker,None,http://store.steampowered.com/app/705860/Space...,None,http://steamcommunity.com/app/705860/reviews/?...,"[Single-player, HTC Vive, Oculus Rift, Tracked...",Free,1.0,705860,None,NaN
120395,None,"[Video Production, Utilities, Web Publishing]",LIV Client,None,http://store.steampowered.com/app/755540/LIV_C...,None,http://steamcommunity.com/app/755540/reviews/?...,"[Steam Workshop, Steam Cloud, HTC Vive, Oculus...",None,0.0,755540,None,NaN


La columna Publisher y la columna developer contienen casi la misma informacion, por ello se precede traer los valores que no estan en publisher pero si en developer

In [25]:
df_steam_games_clean['publisher'].fillna(df_steam_games_clean['developer'], inplace=True)#Remplaza los valores nulos de la columna publisher con los valores de la columna developer

C:\Users\Willc\AppData\Local\Temp\ipykernel_20260\952864719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_clean['publisher'].fillna(df_steam_games_clean['developer'], inplace=True)#Remplaza los valores nulos de la columna publisher con los valores de la columna developer


Ahora verificamos los datos de la columna price

In [26]:
df_steam_games_clean['price'].value_counts()

price
4.99           4256
9.99           3893
2.99           3423
0.99           2578
1.99           2523
               ... 
74.76             1
32.99             1
14.95             1
26.99             1
Free to Use       1
Name: count, Length: 162, dtype: int64

In [27]:
rows_not_number = df_steam_games_clean[df_steam_games_clean['price'].apply(lambda x: not isinstance(x, float))]

In [28]:
rows_not_number['price'].value_counts()

price
Free                             902
Free to Play                     519
Free To Play                     462
Free Mod                           4
Free Demo                          3
Play Now                           2
Third-party                        2
Play for Free!                     2
Play WARMACHINE: Tactics Demo      1
Install Theme                      1
Install Now                        1
Free HITMAN™ Holiday Pack          1
Play the Demo                      1
Starting at $499.00                1
Starting at $449.00                1
Free to Try                        1
Free Movie                         1
Free to Use                        1
Name: count, dtype: int64

In [29]:
df_steam_games_clean['price'] = df_steam_games_clean['price'].replace(['f', 'F', 'I', 'T', 'P'], 0, regex=True)#Reemplazamos los valores que contengan string por 0

C:\Users\Willc\AppData\Local\Temp\ipykernel_20260\1302885621.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_clean['price'] = df_steam_games_clean['price'].replace(['f', 'F', 'I', 'T', 'P'], 0, regex=True)#Reemplazamos los valores que contengan string por 0


Para las filas que contienen Starting at $499.00  y Starting at $449.00 realizamos la siguiente funcion para que el valor de la columna price sea la parte float

In [30]:
# Definir una función para extraer la parte numérica de una cadena y sobrescribir la columna "price"
def replace_nume_price(string):
    if isinstance(string, str) and string.startswith('Starting at $'):
        numeric_part = string[len('Starting at $'):]
        try:
            numeric_value = float(numeric_part)
            return numeric_value
        except ValueError:
            return string  # Si no se puede convertir a float, devolver la cadena original
    else:
        return string  # Si no es una cadena que comienza con "Starting at $", no hacer cambios

In [31]:
# Aplicar la función a la columna "price" para sobrescribir los valores con la parte numérica extraída
df_steam_games_clean['price'] = df_steam_games_clean['price'].apply(replace_nume_price)

C:\Users\Willc\AppData\Local\Temp\ipykernel_20260\1689708497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_clean['price'] = df_steam_games_clean['price'].apply(replace_nume_price)


In [32]:
df_steam_games_clean['price'] = df_steam_games_clean['price'].astype(float)#Cambiamos a tipo de dato Float

C:\Users\Willc\AppData\Local\Temp\ipykernel_20260\1060508159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_clean['price'] = df_steam_games_clean['price'].astype(float)#Cambiamos a tipo de dato Float


Se eliminan las columnas que no se usaran en el analisis

In [33]:
df_steam_games_clean.drop(columns=['developer','url','reviews_url','early_access','specs'], inplace=True)

C:\Users\Willc\AppData\Local\Temp\ipykernel_20260\1958662482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_steam_games_clean.drop(columns=['developer','url','reviews_url','early_access','specs'], inplace=True)


Se verifica que no hay valores nulos en el df

In [34]:
df_steam_games_clean.isnull().sum()

publisher       3209
genres             0
app_name           1
title           2048
release_date    2065
price           1356
id                 0
year            2239
dtype: int64

se guarda como csv y se comprime para que ocupe menos espacio

In [36]:
df_steam_games_clean.to_csv('../datasets/steam_games_worked.csv.gz',index=False)